# CNN for classifying letters 
By: Hesham Asem

______

here we'll build a Conv2d to be used in reading & classifying about half million pictures of  first 10 alphabetic letters . . 

you can find data file  here :  https://www.kaggle.com/jwjohnson314/notmnist

let;s first import libraries 


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras
import os
import glob as gb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten ,Conv2D, MaxPooling2D

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

then we need to check the folders to know what letters available . 

In [2]:
all_letters = os.listdir('../input/notmnist/notMNIST_large/notMNIST_large')

print(f'We have {len(all_letters)} letters , which are : {all_letters}')

We have 10 letters , which are : ['A', 'D', 'I', 'G', 'H', 'B', 'F', 'C', 'J', 'E']


____


# Read Data . 

we'll use glob library to collect all png pictures to know how many pictures we have for each letter . 

In [3]:
total_images = 0
for letter in all_letters : 
    available_images = gb.glob(pathname= f'../input/notmnist/notMNIST_large/notMNIST_large/{letter}/*.png')
    total_images+=len(available_images)
    print(f'for letter {letter} we have  {len(available_images)} available images')
print('-----------------------')    
print(f'Total Images are {total_images} images')

for letter A we have  52912 available images
for letter D we have  52912 available images
for letter I we have  52912 available images
for letter G we have  52912 available images
for letter H we have  52912 available images
for letter B we have  52912 available images
for letter F we have  52912 available images
for letter C we have  52912 available images
for letter J we have  52911 available images
for letter E we have  52912 available images
-----------------------
Total Images are 529119 images


total 529 thousand images for all 10 letters , now let's create X & y variables , so we can fill them with read data


In [4]:
X = list(np.zeros(shape=(total_images , 28,28)))
y = list(np.zeros(shape=(total_images)))

now to open each file & read it using plt.imread , then fill it in its place in X & y data

In [5]:
i=0
y_value = 0
for letter in all_letters : 
    available_images = gb.glob(pathname= f'../input/notmnist/notMNIST_large/notMNIST_large/{letter}/*.png')
    for image in available_images : 
        try : 
            x = plt.imread(image)
            X[i] = x
            y[i] = y_value
            i+=1
        except : 
            pass
    y_value+=1

____

# Forming Dimensions

since (y) data now is a single number vary from 0 to 9 , we'll need to categorize it using OneHotEncoder from sklearn , so it be ready for the softmax activation functing in CNN

In [6]:
ohe  = OneHotEncoder()
y = np.array(y)
y = y.reshape(len(y), 1)
ohe.fit(y)
y = ohe.transform(y).toarray()

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


now we can check a random y value

In [7]:
y[10000]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

then we have to expand X dimension to be suitable with the CNN dimensions

In [8]:
X = np.expand_dims(X, -1).astype('float32')/255.0

now X shape should be : sample size * 28 * 28 * 1

In [9]:
X.shape

(529119, 28, 28, 1)

# Splitting Data

now lets split our dat to Train , Cross-Validation & Test sets . . 

first to create X_part & y_part which is 85% of data , also X_test & y_test which is 15%


In [10]:
X_part, X_cv, y_part, y_cv = train_test_split(X, y, test_size=0.15, random_state=44, shuffle =True)

print('X_train shape is ' , X_part.shape)
print('X_test shape is ' , X_cv.shape)
print('y_train shape is ' , y_part.shape)
print('y_test shape is ' , y_cv.shape)

X_train shape is  (449751, 28, 28, 1)
X_test shape is  (79368, 28, 28, 1)
y_train shape is  (449751, 10)
y_test shape is  (79368, 10)


then to split X_part & y_part into train & test

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_part, y_part, test_size=0.25, random_state=44, shuffle =True)

print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

X_train shape is  (337313, 28, 28, 1)
X_test shape is  (112438, 28, 28, 1)
y_train shape is  (337313, 10)
y_test shape is  (112438, 10)


# Build the Model

now let's build the model with Keras , using Conv2d & Maxpooling tools , & not to forget to dropout some cells to avoid OF

In [12]:
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(filters = 32, kernel_size = 4,  activation = tf.nn.relu , padding = 'same'),
        keras.layers.MaxPool2D(pool_size=(3,3), strides=None, padding='valid'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=32, kernel_size=4,activation = tf.nn.relu , padding='same'),
        keras.layers.MaxPool2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=64, kernel_size=5,activation = tf.nn.relu , padding='same'),
        keras.layers.MaxPool2D(),
        keras.layers.Flatten(),    
        keras.layers.Dropout(0.5),        
        keras.layers.Dense(64),    
        keras.layers.Dropout(0.3),            
        keras.layers.Dense(units= 10,activation = tf.nn.softmax ),                

    ])
    

KerasModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

then to train it , using few number of epochs to avoid OF

In [13]:
#Train
KerasModel.fit(X_train,y_train,validation_data=(X_cv, y_cv),epochs=3,batch_size=64,verbose=1)

Train on 337313 samples, validate on 79368 samples
Epoch 1/3
337313/337313 [==============================] - 42s 125us/step - loss: 0.4316 - acc: 0.8713 - val_loss: 0.3589 - val_acc: 0.8976
Epoch 2/3
337313/337313 [==============================] - 39s 115us/step - loss: 0.3442 - acc: 0.8969 - val_loss: 0.3090 - val_acc: 0.9096
Epoch 3/3
337313/337313 [==============================] - 38s 113us/step - loss: 0.3191 - acc: 0.9034 - val_loss: 0.2927 - val_acc: 0.9112


now how the model looks like ? 

In [14]:
KerasModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        544       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 32)          128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          16416     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 32)          128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          51264     
__________

# Predicting

then we'll predict X_test

In [15]:
y_pred = KerasModel.predict(X_test)

print('Prediction Shape is {}'.format(y_pred.shape))

Prediction Shape is (112438, 10)


and we can check random samples from X_test

In [16]:
Letters ={0:'A', 1:'B' , 2:'C' ,3:'D' ,4:'E' ,5:'F' ,6:'G' ,7:'H' ,8:'I' ,9:'J' }

for i in list(np.random.randint(0,len(X_test) ,size= 10)) : 
    print(f'for sample  {i}  the predicted value is   {Letters[np.argmax(y_pred[i])]}   , while the actual letter is {Letters[np.argmax(y_test[i])]}')


for sample  98096  the predicted value is   F   , while the actual letter is F
for sample  103877  the predicted value is   I   , while the actual letter is H
for sample  10249  the predicted value is   D   , while the actual letter is D
for sample  97628  the predicted value is   H   , while the actual letter is H
for sample  95360  the predicted value is   C   , while the actual letter is C
for sample  41433  the predicted value is   D   , while the actual letter is D
for sample  74535  the predicted value is   C   , while the actual letter is C
for sample  68969  the predicted value is   F   , while the actual letter is F
for sample  108700  the predicted value is   B   , while the actual letter is B
for sample  81442  the predicted value is   C   , while the actual letter is I


& to measure the loss & accuracy

In [17]:
ModelLoss, ModelAccuracy = KerasModel.evaluate(X_test, y_test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

112438/112438 [==============================] - 7s 58us/step
Test Loss is 0.2878474014839032
Test Accuracy is 0.9132766502417402


great , with 91% accuracy we achieved good result without moving to OF